In [1]:
from typing import *

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python import keras
from tensorflow.python.keras import callbacks
from tensorflow.keras.utils import Sequence as KerasSequence
from sklearn.utils import shuffle as sk_shuffle

import os
from datetime import datetime

from lib.dnn.utils import KerasTrainer
from core.utils.file_storage import DropboxClient
from lib.dnn.layers import Delta, MovingAverage, Percentage


2022-03-03 16:22:48.032301: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-03 16:22:48.032353: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Couldn't Connect to Postgres could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?



In [2]:
TRAIN_PATH = "Data/Processed/train"
TEST_PATH = "Data/Processed/test"
TRAIN_FILES = [os.path.join(TRAIN_PATH, path) for path in os.listdir(TRAIN_PATH)][:3]
TEST_FILES = [os.path.join(TEST_PATH, path) for path in os.listdir(TEST_PATH)]
MAX_ROW = 1000000

MODEL_OUT_PATH = "model.h5"
BATCH_SIZE = 64
EPOCHS = 50
TRAINER_CONFIG = {
	"seq_len": 64,
	"hidden_layers": [1024, 2048, 1024, 64, 32],
	"delta": True,
	"percentage": True,
	"hidden_activation": "relu",
	"optimizer": "adam",
	"loss": "binary_crossentropy",
	"average_window": 0
}
CALLBACKS = []


In [3]:
class DataGenerator(KerasSequence):
	
	def __init__(self, files: List[str], batch_size: int, max_row: int = 1000000, y_column_header: str = "y"):
		self.__files = sorted(files)
		self.__batch_size = batch_size
		self.__max_row = max_row
		self.__y_column_header = y_column_header
		self.__cache = {}
		self.__size = int(np.ceil(self.__get_total_num_rows()/self.__batch_size))
		self.__random_state = self.__get_new_random_state()
	
	def __load_file(self, idx, shuffle=True) -> pd.DataFrame:
		cached = self.__cache.get(str(idx))
		if cached is not None:
			return cached
		df = pd.read_csv(self.__files[idx])
		self.__cache.clear()
		self.__cache[str(idx)] = df
		
		if shuffle:
			df = sk_shuffle(df, random_state=self.__random_state)
			df.reset_index(drop=True, inplace=True)
		
		return df
	
	def __separate_input_output(self, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
		return df.drop(columns=[self.__y_column_header]).to_numpy(), df[self.__y_column_header].to_numpy()
	
	def __get_indexes(self, idx: int) -> Tuple[int, int]:
		return idx // self.__max_row, idx % self.__max_row
	
	def __get_slice(self, start: int, end: int) -> pd.DataFrame:
		start_file_index, start_row_index = self.__get_indexes(start)
		end_file_index, end_row_index = self.__get_indexes(end)
		
		if start_file_index == end_file_index:
			return self.__load_file(start_file_index).iloc[start_row_index: end_row_index]
		
		batch_start = self.__load_file(start_file_index).iloc[start_file_index:]
		if end_file_index > len(self.__files) - 1:
			return batch_start
		batch_end = self.__load_file(end_file_index).iloc[:end_file_index]
		return batch_start.append(batch_end)

	def __getitem__(self, index):
		return self.__separate_input_output(
			self.__get_slice(
				self.__batch_size * index,
				self.__batch_size * (index + 1)
			)
		)
	
	def __get_total_num_rows(self) -> int:
		last_file_size = len(self.__load_file(len(self.__files) - 1, shuffle=False))
		return (self.__max_row * len(self.__files[:-1]))+last_file_size

	def __get_new_random_state(self) -> int:
		return np.random.randint(len(self.__files)*100)

	def on_epoch_end(self):
		self.__random_state = self.__get_new_random_state()

	def __len__(self):
		return self.__size


In [5]:
class ModelCheckpointCallback(callbacks.Callback):
	
	def __init__(self):
		super(ModelCheckpointCallback, self).__init__()
		self.__dropbox_client = DropboxClient()
	
	def __save_model(self, model: keras.Model) -> str:
		file_name = f"{datetime.now().timestamp()}.h5"
		model.save(file_name)
		return file_name
		
	def __upload_model(self, model_path: str):
		self.__dropbox_client.upload_file(model_path)
	
	def on_epoch_end(self, epoch, logs=None):
		print("[+]Saving Model...")
		path = self.__save_model(self.model)
		print("[+]Uploading Model...")
		self.__upload_model(path)


In [6]:
class Trainer:
	def __init__(self, 
					seq_len: int,
					hidden_layers: List[int], 
					train_files: List[str],
					test_files: List[str],
					model_out_path: str,
					loss="binary_crossentropy",
					optimizer="adam",
					hidden_activation="relu",
					delta=True,
					average_window=None,
					percentage=True,
					batch_size: int = 1024,
					epochs: int = 50,
					callbacks: List[callbacks.Callback] = None
					):
		self.__seq_len = seq_len
		self.__hidden_layers = hidden_layers
		self.__loss = loss
		self.__optimizer = optimizer
		self.__hidden_activation = hidden_activation
		self.__delta = delta
		self.__percentage = percentage
		self.__average_window = average_window
		if average_window is not None:
			self.__average_window = int(self.__seq_len * average_window)
		self.__model_out_path = model_out_path
		self.__train_files = train_files
		self.__test_files = test_files
		self.__batch_size = batch_size
		self.__epochs = epochs
		self.__callbacks = callbacks
		if callbacks is None:
			self.__callbacks = []

	def __create_generator(self, files: List[str], batch_size: int):
		return DataGenerator(files, batch_size)

	def __create_model(self) -> keras.Model:
		input_layer = keras.layers.Input(shape=self.__seq_len)

		pre_hidden_layer = input_layer

		if self.__average_window != 0:
			pre_hidden_layer = MovingAverage(self.__average_window)(pre_hidden_layer)

		pre_hidden_layer_copy = pre_hidden_layer[:, : -1]

		if self.__delta:
			pre_hidden_layer = Delta()(pre_hidden_layer)
		else:
			pre_hidden_layer = pre_hidden_layer[:, 1:]

		if self.__percentage:
			pre_hidden_layer = tf.concat(
				(
					tf.reshape(pre_hidden_layer, (-1, 1, pre_hidden_layer.shape[1])),
					tf.reshape(pre_hidden_layer_copy, (-1, 1, pre_hidden_layer.shape[1]))
				),
				axis=1
			)
			pre_hidden_layer = Percentage()(pre_hidden_layer)

		output_layer = keras.layers.Dense(1, activation="sigmoid")

		if len(self.__hidden_layers) == 0:
			output_layer = output_layer(pre_hidden_layer)
		else:
			hidden_layer = keras.layers.Dense(self.__hidden_layers[0], activation=self.__hidden_activation)(
				pre_hidden_layer)
			for layer_size in self.__hidden_layers[1:]:
				hidden_layer = keras.layers.Dense(layer_size, activation=self.__hidden_activation)(hidden_layer)
			output_layer = output_layer(hidden_layer)

		model = keras.Model(inputs=input_layer, outputs=output_layer)

		return model

	def __compile_model(self, model: keras.Model):
		model.compile(loss=self.__loss, optimizer=self.__optimizer, metrics=["accuracy"])
	
	def start(self):
		print("[+]Starting Training....")
		
		train_generator = self.__create_generator(self.__train_files, self.__batch_size)
		test_generator = self.__create_generator(self.__test_files, self.__batch_size)
		
		model = self.__create_model()
		self.__compile_model(model)
		print("[+]Fitting Model...")
		model.fit(train_generator, epochs=self.__epochs, callbacks=self.__callbacks, verbose=True, shuffle=False)
		print("[+]Evaluating Model...")
		model.evaluate(test_generator)
		print("[+]Saving Model...")
		model.save(self.__model_out_path)
		print("[+]Done")


In [7]:
trainer = Trainer(
	train_files=TRAIN_FILES,
	test_files=TEST_FILES,
	model_out_path=MODEL_OUT_PATH,
	**TRAINER_CONFIG,
	batch_size=BATCH_SIZE,
	epochs=EPOCHS,
	callbacks=CALLBACKS,
)
trainer.start()


[+]Starting Training....


2022-03-03 16:23:33.241308: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-03 16:23:33.241348: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-03 16:23:33.241385: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dhcppc7): /proc/driver/nvidia/version does not exist
2022-03-03 16:23:33.241779: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[+]Fitting Model...


2022-03-03 16:23:46.197648: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-03 16:23:46.247062: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 1999965000 Hz


Epoch 1/50


...

 4711/46875 [==>...........................] - ETA: 26:46 - loss: 0.6921 - accuracy: 0.5237

 4713/46875 [==>...........................] - ETA: 26:46 - loss: 0.6921 - accuracy: 0.5237

 4715/46875 [==>...........................] - ETA: 26:46 - loss: 0.6921 - accuracy: 0.5237

 4716/46875 [==>...........................] - ETA: 26:46 - loss: 0.6921 - accuracy: 0.5237

KeyboardInterrupt: 

In [15]:
generator = DataGenerator(["Data/Processed/train/1646061243.894905.csv"], 32)

generator[5]


(array([[0.94142  , 0.94146  , 0.94144  , ..., 0.94196  , 0.94196  ,
         0.94193  ],
        [0.9223   , 0.92254  , 0.9225   , ..., 0.9223   , 0.92233  ,
         0.922    ],
        [0.93323  , 0.9334405, 0.9334843, ..., 0.93325  , 0.9332984,
         0.9330505],
        ...,
        [0.91681  , 0.91679  , 0.91688  , ..., 0.91705  , 0.91705  ,
         0.91695  ],
        [0.8959   , 0.89595  , 0.89591  , ..., 0.89686  , 0.89686  ,
         0.89709  ],
        [0.85295  , 0.85285  , 0.85232  , ..., 0.8534   , 0.85357  ,
         0.8534   ]]),
 array([0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
        1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0.]))

In [16]:
generator[6]

(array([[0.9363986, 0.9366977, 0.93697  , ..., 0.93673  , 0.9363   ,
         0.9362   ],
        [0.9335   , 0.93343  , 0.93338  , ..., 0.93399  , 0.93398  ,
         0.93405  ],
        [0.88473  , 0.88494  , 0.88494  , ..., 0.88606  , 0.88598  ,
         0.88598  ],
        ...,
        [0.90155  , 0.90152  , 0.90151  , ..., 0.90168  , 0.9017   ,
         0.9017   ],
        [0.93422  , 0.93421  , 0.9341   , ..., 0.9338551, 0.934    ,
         0.934    ],
        [0.9220215, 0.92203  , 0.92203  , ..., 0.9215401, 0.9209   ,
         0.92144  ]]),
 array([0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1.]))